In [1]:
import subprocess
from urllib.parse import urljoin

import pandas as pd
import requests
from lxml import html
import js2xml

In [2]:
TARGET_COLUMNS = [
    'hospital_id',
    #'row_id',
    'line_type',
    'description',
    'rev_code',
    'local_code',
    'code',
    'ms_drg',
    'apr_drg',
    'eapg',
    'hcpcs_cpt',
    'modifiers',
    'alt_hcpcs_cpt',
    'thru',
    'apc',
    'icd',
    'ndc',
    'drug_hcpcs_multiplier',
    'drug_quantity',
    'drug_unit_of_measurement',
    'drug_type_of_measurement',
    'billing_class',
    'setting',
    'payer_category',
    'payer_name',
    'plan_name',
    'standard_charge',
    'standard_charge_percent',
    'contracting_method',
    'additional_generic_notes',
    'additional_payer_specific_notes'
]

In [3]:
# https://www.adventisthealth.org/patient-resources/financial-services/healthcare-costs-and-charges/cost-estimator-tool/
ccn = "050455"
app_url = "https://apps.para-hcfs.com/PTT/FinalLinks/Adventist_Bakersfield_V3.aspx"

In [4]:
resp = requests.get(app_url)
resp

<Response [200]>

In [5]:
tree = html.fromstring(resp.text)
js_link = tree.xpath('//script[contains(@src, "/PTT/extnet/extnet-init-js")]/@src')[0]
js_url = urljoin(resp.url, js_link)
resp1 = requests.get(js_url)
parsed = js2xml.parse(resp1.text)
db_name = parsed.xpath('//object[./property/string[text()="App.hdnDB_Container"]]/property[@name="value"]/string/text()')[0]
db_name

'dbAHBBAKERSFIELDCA'

In [6]:
params = {
    'dbName': db_name,
    'type': 'CDMWithoutLabel'
}

headers = {
    'authority': 'apps.para-hcfs.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
    'cache-control': 'no-cache',
    'content-type': 'multipart/form-data; boundary=----WebKitFormBoundarysVqstz3xq11k5yBT',
    'origin': 'https://apps.para-hcfs.com',
    'pragma': 'no-cache',
    'referer': resp.url,
    'sec-ch-ua': '"Google Chrome";v="113", "Chromium";v="113", "Not-A.Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'iframe',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
}

data = '------WebKitFormBoundarysVqstz3xq11k5yBT\r\nContent-Disposition: form-data; name="__EVENTTARGET"\r\n\r\nResourceManager\r\n------WebKitFormBoundarysVqstz3xq11k5yBT\r\nContent-Disposition: form-data; name="__EVENTARGUMENT"\r\n\r\n-|public|DownloadReport\r\n------WebKitFormBoundarysVqstz3xq11k5yBT\r\nContent-Disposition: form-data; name="__ExtNetDirectEventMarker"\r\n\r\ndelta=true\r\n------WebKitFormBoundarysVqstz3xq11k5yBT--\r\n'

resp2 = requests.post('https://apps.para-hcfs.com/PTT/FinalLinks/Reports.aspx', params=params, headers=headers, data=data)
print(resp2)

<Response [200]>


In [7]:
filename = resp2.headers['content-disposition'].split('"')[1]
filename

'952294234_adventist-health---bakersfield_standardcharges.csv'

In [8]:
csv_f = open(filename, "w")
csv_f.write(resp2.text)
csv_f.close()

In [10]:
idx1 = resp2.text.index("Procedure Code,Procedure Description")
idx2 = resp2.text.index("\r\n\r\nShoppable Services Code,Shoppable Services Description")
idx1, idx2

(61, 34952907)

In [14]:
csv_str = resp2.text[idx1:idx2]

from io import StringIO

csv_buf = StringIO(csv_str)

df_in = pd.read_csv(csv_buf, dtype=str)
df_in

,Procedure Code,Procedure Description,Price Tier,Revenue Code,CPT HCPCS Code,Modifier1,Modifier2,NDC Code,Rx Unit Multiplier,Gross Charge,...,KAISER_PPO_POS_-_ALL_OTHER_PLANS,KERN_LEGACY_HP_EPO_-_ALL_OTHER_PLANS,KERN_LEGACY_SHARE_SELECT,OSCAR_-_ALL_PLANS,PHCS-_ALL_PLANS,THREE_RIVERS_PROVIDER_NETWORK-_ALL_PLANS,UHC_ALL_PAYER_-_ALL_OTHER_PLANS,UHC_HMO,UHC_JLL_CSP,WESTERN_GROWERS_ASSURANCE-_ALL_PLANS
0,1000006_1,Culture Wound wo Gram Stain,Inpatient,0306,87070,NaN,NaN,NaN,0,325.00,...,198.25,NaN,NaN,88.92,NaN,260.00,NaN,NaN,NaN,NaN
1,1000006_1,Culture Wound wo Gram Stain,Outpatient,0306,87070,NaN,NaN,NaN,0,325.00,...,198.25,162.50,162.50,13.79,211.25,260.00,10.34,10.34,10.34,149.50
2,1004056_1,Bill Serological Method 87147,Outpatient,0306,87147,NaN,NaN,NaN,0,99.00,...,60.39,49.50,49.50,8.29,64.35,79.20,6.22,6.22,6.22,45.54
3,1004056_1,Bill Serological Method 87147,Inpatient,0306,87147,NaN,NaN,NaN,0,99.00,...,60.39,NaN,NaN,27.09,NaN,79.20,NaN,NaN,NaN,NaN
4,1004058_1,Bill Mold Definitive ID 87107,Inpatient,0306,87107,NaN,NaN,NaN,0,115.00,...,70.15,NaN,NaN,31.46,NaN,92.00,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121482,981,EXTENSIVE O.R. PROCEDURES UNRELATED TO PRINCIP...,Inpatient,NaN,981,NaN,NaN,NaN,NaN,214753.0230,...,130999.34,38941.50,36461.50,62031.58,29400.00,171802.42,87487.46,65863.90,83111.10,55056.51
121483,982,EXTENSIVE O.R. PROCEDURES UNRELATED TO PRINCIP...,Inpatient,NaN,982,NaN,NaN,NaN,NaN,193489.6350,...,118028.68,33467.75,30667.75,33968.13,31750.00,154791.71,44738.76,33364.08,42501.45,58884.45
121484,983,EXTENSIVE O.R. PROCEDURES UNRELATED TO PRINCIP...,Inpatient,NaN,983,NaN,NaN,NaN,NaN,112140.1000,...,68405.46,2500.00,2250.00,22619.23,3100.00,89712.08,29791.36,22217.00,28301.54,4648.00
121485,987,NON-EXTENSIVE O.R. PROCEDURES UNRELATED TO PRI...,Inpatient,NaN,987,NaN,NaN,NaN,NaN,166288.4179,...,101435.93,31834.01,29604.01,44940.53,22450.00,133030.73,59190.30,44141.36,56230.29,47451.02
